<a href="https://colab.research.google.com/github/Granero0011/Labs16/blob/master/airbnbmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Display Options

In [0]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import scipy.optimize.nnls as nnls

In [0]:
pd.set_option('display.max_rows', 20000)
pd.set_option('display.max_columns', 20000)

In [8]:
from google.colab import files

uploaded = files.upload()

Saving listings.csv to listings.csv


In [9]:
import io
df2 = pd.read_csv(io.BytesIO(uploaded['listings.csv']))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Dataframes

In [4]:
#df = pd.read_csv("listings.csv")
#df2 = pd.read_csv("listings.csv.gz")               

FileNotFoundError: ignored

In [0]:
#features used in model
features = ['id', 'bedrooms', 'bathrooms', 'amenities', 'price']
#model dataframe
model_df = df2[features]
#model data, remove empty amenities
model_data = model_df[(model_df['amenities'] != '{}') & (model_df['bedrooms'] != 0) & (model_df['bathrooms'] != 0)]

# Helper functions

In [11]:
model_data.head()

,id,bedrooms,bathrooms,amenities,price
0,2818,1.0,1.5,"{Internet,Wifi,""Paid parking off premises"",""Bu...",$59.00
1,20168,1.0,1.0,"{TV,Internet,Wifi,""Paid parking off premises"",...",$80.00
2,25428,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,Kitchen,Elevator,...",$125.00
3,27886,1.0,1.0,"{TV,Internet,Wifi,Breakfast,Heating,""Smoke det...",$150.00
4,28871,1.0,NaN,"{Internet,Wifi,""Pets live on this property"",Ca...",$75.00


### Function that changes the amenities data in the dataframe from json to a list

In [0]:
def json_to_list(df):
    new_data = []
    data = df['amenities'].values
    for num, amen in enumerate(data):
        new_data.append([word.strip('"') for word in amen.strip("{").strip("}").split(",")])
    df['new_amenities'] = new_data
    return df

In [13]:
model_data = json_to_list(model_data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Function that figures out the unique amenities

In [0]:
def unique_amenities(df, column):
    uniques = []
    for data in df[column]:
        for d in data:
            if d in uniques:
                continue
            else:
                uniques.append(d)
    return uniques

In [0]:
amenities = unique_amenities(model_data, 'new_amenities')

In [16]:
amenities

['Internet',
 'Wifi',
 'Paid parking off premises',
 'Buzzer/wireless intercom',
 'Heating',
 'Washer',
 'Smoke detector',
 'Carbon monoxide detector',
 'First aid kit',
 'Safety card',
 'Fire extinguisher',
 'Essentials',
 'Shampoo',
 'Lock on bedroom door',
 '24-hour check-in',
 'Hangers',
 'Hair dryer',
 'Iron',
 'Laptop friendly workspace',
 'translation missing: en.hosting_amenity_49',
 'translation missing: en.hosting_amenity_50',
 'Private entrance',
 'Hot water',
 'Bed linens',
 'Extra pillows and blankets',
 'Single level home',
 'Garden or backyard',
 'No stairs or steps to enter',
 'Flat path to guest entrance',
 'Well-lit path to entrance',
 'Accessible-height bed',
 'Host greets you',
 'Handheld shower head',
 'Paid parking on premises',
 'TV',
 'Refrigerator',
 'Long term stays allowed',
 'Cable TV',
 'Kitchen',
 'Elevator',
 'Indoor fireplace',
 'Family/kid friendly',
 'Dryer',
 'Breakfast',
 'Self check-in',
 'Smart lock',
 'Private living room',
 'Lake access',
 'Pets 

### Function that gathers the counts of every amenity

In [0]:
def amen_counts(amen_lst, df, column):
    counts = [0 for num in range(len(amen_lst))]
    amen_data = df[column].values
    for num, data in enumerate(amen_data):
        for d in data:
            num = amen_lst.index(d)
            counts[num] += 1
    return counts

In [0]:
counts = amen_counts(amenities, model_data, 'new_amenities')

### Function that returns top 20 amenities 

In [0]:
def top_20(counts):
    largest = sorted(counts)[-20:]
    count_indexes = []
    for l in largest:
        count_indexes.append(counts.index(l))
        amens = []
    for c in count_indexes:
        amens.append(amenities[c])
    return largest, amens      

In [0]:
largest, top = top_20(counts)

In [21]:
largest, top

([6641,
  7252,
  7473,
  7745,
  7919,
  8128,
  8791,
  9231,
  12715,
  12779,
  13000,
  13107,
  13792,
  14746,
  15093,
  15358,
  17496,
  18457,
  18548,
  18691],
 ['Internet',
  'First aid kit',
  'Cable TV',
  'Carbon monoxide detector',
  'Family/kid friendly',
  'Fire extinguisher',
  'Dryer',
  'Hot water',
  'Hair dryer',
  'Shampoo',
  'Iron',
  'Laptop friendly workspace',
  'Hangers',
  'Washer',
  'TV',
  'Smoke detector',
  'Kitchen',
  'Essentials',
  'Heating',
  'Wifi'])

### Function that create columns with only the top amenities

In [0]:
def reduce_dimensions(df, column, top):
    all_data = []
    all_strings = []
    amens = df[column].values
    bedrooms = df['bedrooms'].values
    bathrooms = df['bathrooms'].values
    for num, data in enumerate(amens):
        new_data = []
        new_string = ""
        for d in data:
            if d in top:
                new_data.append(d)
                new_string += d + " "
            else:
                continue
        new_string += str(bedrooms[num])
        new_string += str(bathrooms[num])
        all_data.append(new_data)
        all_strings.append(new_string)
    df['Groups'] = all_strings
    df['New_data'] = all_data
    return df

In [23]:
model_data = reduce_dimensions(model_data, "new_amenities", top)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [0]:
model_data = model_data.dropna()

### Grouping system

In [0]:
encoder = OrdinalEncoder()

In [0]:
groups = encoder.fit_transform(model_data[['Groups']].values)

In [0]:
model_data['OrdinalGroups'] = groups

In [0]:
def optimal_pricing(df):
    groups = [word for word in df['OrdinalGroups'].values]
    ids = [word for word in df['id'].values]
    prices = [word for word in df['id'].values]
    for g in groups:
        data = []
        max_df = df[df['OrdinalGroups'] == g]
        max_price = max_df['price'].max()
        group_ids = [word for word in max_df['id'].values]
        indexes = []
        for i in group_ids:
            indexes.append(ids.index(i))
        for index in indexes:
            prices[index] = max_price
    df['price'] = prices
    return df

In [0]:
df = optimal_pricing(model_data)

In [0]:
def string_to_int(df, column):
    new_data = []
    data = df[column].values
    for d in data:
        d = d.strip("$")
        d = d.replace(",", "")
        d = d.split(".")[0]
        d = int(d)
        new_data.append(d)
    new_data = np.array(new_data)
    df[column] = new_data
    return df

In [0]:
df = string_to_int(df, "price")

In [0]:
def onehotencoder(df, column, amen_lst):
    for amen in amen_lst:
        new_data = []
        data = df[column].values
        for d in data:
            if amen in d:
                new_data.append(1)
            else:
                new_data.append(0)
        new_data = np.array(new_data)
        df[amen] = new_data
    return df

In [0]:
df = onehotencoder(df, "New_data",  top)

# Model

In [0]:
features = top + ["bedrooms", "bathrooms"]
target = 'price'
draft_features = df[features]
draft_target = df[target]

In [35]:
draft_features.shape, draft_target.shape

((19134, 22), (19134,))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(draft_features, draft_target, train_size = .8, test_size = .2, random_state= 42)

In [37]:
X_train.shape, y_train.shape

((15307, 22), (15307,))

In [38]:
model = LinearRegression()
model.fit(draft_features, draft_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
def model_coefs():
    coefs = model.coef_
    positive_c = []
    negative_c = []
    positive_f = []
    negative_f = []
    for num, coef in enumerate(coefs):
        if coef > 0:
            positive_c.append(coef)
            positive_f.append(features[num])
        else:
            negative_c.append(coef)
            negative_f.append(features[num])
    print("Positives")
    for num, p in enumerate(positive_c):
        print(f"Feature: {positive_f[num]}, Coef: {p}")
    print("Negatives")
    for num, p in enumerate(negative_c):
        print(f"Feature: {negative_f[num]}, Coef: {p}")
    return

In [40]:
model_coefs()

Positives
Feature: First aid kit, Coef: 0.22997663736152418
Feature: Cable TV, Coef: 12.052372914670153
Feature: Carbon monoxide detector, Coef: 5.467200326838577
Feature: Family/kid friendly, Coef: 1.8739322485326704
Feature: Fire extinguisher, Coef: 14.830987898157945
Feature: Dryer, Coef: 23.12005032741777
Feature: Hot water, Coef: 8.983016371883203
Feature: Hair dryer, Coef: 5.504567545738264
Feature: Iron, Coef: 3.8152160092359515
Feature: Hangers, Coef: 3.388545635658039
Feature: TV, Coef: 18.78189990481452
Feature: Kitchen, Coef: 15.464231517121203
Feature: Wifi, Coef: 8.096428109944009
Feature: bedrooms, Coef: 54.509755657319474
Feature: bathrooms, Coef: 41.82568412103905
Negatives
Feature: Internet, Coef: -5.7438231137574105
Feature: Shampoo, Coef: -2.6463070358145337
Feature: Laptop friendly workspace, Coef: -3.9505129092879714
Feature: Washer, Coef: -21.1904417066778
Feature: Smoke detector, Coef: -0.0757402512441457
Feature: Essentials, Coef: -48.16807099631269
Feature: Hea

In [0]:
x, rnorm = nnls(X_train,y_train)

In [42]:
print(x)

[ 0.          0.         10.62694565  3.44705141  0.         14.54643802
 15.72702677  6.51512023  2.30820011  0.          0.          0.
  0.          0.         15.60455762  0.          0.          0.
  0.          0.         53.55775886 40.01312475]
